In [1]:
import logging
import pandas as pd
import numpy as np
from numpy import random
import gensim
import nltk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
import re
from bs4 import BeautifulSoup
%matplotlib inline

df_train = pd.read_csv('train.csv')
print(df_train.head(10))

print("\n\n\n")

df_test = pd.read_csv("test.csv")
print(df_test.head(10))


df_mixed_train = pd.read_csv("train_mixed.csv")
print(df_test.head(10))

                                            headline   category
0                      Myanmar'da çatışmalar sürüyor      dunya
1  Ermenistan'da cumhurbaşkanı adayına silahlı sa...      dunya
2                             Süper Lig'den transfer       spor
3           Erdoğan: İç hesaplaşma ihtimali yüksek    siyaset
4                      AB'den güvenli internet adımı  teknoloji
5                AB "1 trilyon euro" için toplanıyor    ekonomi
6                    İpek Şenoğlu Cup 11-24 Şubatta       spor
7        Kordsa 120 iş liderini İstanbulda ağırladı    ekonomi
8       Bursada 900 milyon ton mermer rezervi çıktı    ekonomi
9               'Başbakan tek otorite olmak istiyor'    siyaset




                                     headline category
0   İsrail'den Gazze Şeridi'ne hava saldırısı    dunya
1     İdam edecek ama organlara el koymayacak    dunya
2              4 milyon dolar nafaka ödeyecek    dunya
3   Mahmud Abbas'ın 'Bağımsız Filistin' umudu    dunya
4              Br

In [2]:
x_train = df_train.headline
y_train = df_train.category

x_test= df_test.headline
y_test = df_test.category

x_mixed_train = df_mixed_train.headline
y_mixed_train = df_mixed_train.category

x_mixed_test= df_test.headline
y_test = df_test.category


In [9]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer

nb = Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clf', MultinomialNB()),
              ])
nb.fit(x_train, y_train)

from sklearn.metrics import classification_report
y_pred = nb.predict(x_test)

print("------Naive Bayes, base training------")
my_categories = ["dunya", "ekonomi","spor","siyaset","teknoloji","kultur","saglik"]
print('accuracy %s' % accuracy_score(y_pred, y_test))
NB_base_report = classification_report(y_test, y_pred,target_names=my_categories)
print(NB_base_report)

------Naive Bayes, base training------
accuracy 0.6610820780817707
              precision    recall  f1-score   support

       dunya       0.67      0.76      0.71      1489
     ekonomi       0.84      0.37      0.52       903
        spor       1.00      0.03      0.06       298
     siyaset       0.97      0.18      0.31       373
   teknoloji       0.93      0.19      0.32       498
      kultur       0.63      0.97      0.76      2729
      saglik       1.00      0.01      0.03       216

    accuracy                           0.66      6506
   macro avg       0.86      0.36      0.39      6506
weighted avg       0.74      0.66      0.60      6506



In [6]:
df_mixed_default_train = pd.read_csv("train_default_mixed.csv")


x_mixed_default_train = df_mixed_default_train.headline
y_mixed_default_train = df_mixed_default_train.category
nb = Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clf', MultinomialNB()),
              ])
nb.fit(x_mixed_default_train, y_mixed_default_train)

from sklearn.metrics import classification_report
y_pred = nb.predict(x_test)

print("------Naive Bayes, data augmented with default gpt2------")

print('accuracy %s' % accuracy_score(y_pred, y_test))
NB_default_augmented_report = classification_report(y_test, y_pred,target_names=my_categories)
print(NB_default_augmented_report)


------Naive Bayes, data augmented with default gpt2------
accuracy 0.6801414079311405
              precision    recall  f1-score   support

       dunya       0.76      0.67      0.71      1489
     ekonomi       0.52      0.79      0.63       903
        spor       1.00      0.02      0.03       298
     siyaset       1.00      0.10      0.18       373
   teknoloji       0.94      0.15      0.26       498
      kultur       0.70      0.95      0.81      2729
      saglik       1.00      0.00      0.01       216

    accuracy                           0.68      6506
   macro avg       0.85      0.38      0.38      6506
weighted avg       0.75      0.68      0.62      6506



In [12]:
nb = Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clf', MultinomialNB()),
              ])
nb.fit(x_mixed_train, y_mixed_train)

from sklearn.metrics import classification_report
y_pred = nb.predict(x_mixed_test)

print("------Naive Bayes, data augmented training------")

my_categories = ["dunya", "ekonomi","spor","siyaset","teknoloji","kultur","saglik"]
print('accuracy %s' % accuracy_score(y_pred, y_test))
NB_augmented_report = classification_report(y_test, y_pred,target_names=my_categories)
print(NB_augmented_report)

------Naive Bayes, data augmented training------
accuracy 0.692130341223486
              precision    recall  f1-score   support

       dunya       0.74      0.71      0.72      1489
     ekonomi       0.59      0.78      0.67       903
        spor       1.00      0.02      0.04       298
     siyaset       0.98      0.13      0.23       373
   teknoloji       0.93      0.17      0.29       498
      kultur       0.70      0.96      0.81      2729
      saglik       1.00      0.01      0.02       216

    accuracy                           0.69      6506
   macro avg       0.85      0.40      0.40      6506
weighted avg       0.75      0.69      0.63      6506



In [17]:
print("base\n")
print(NB_base_report)
print("------------------------------------------")
print("data augmented with baseline gpt2\n")
print(NB_default_augmented_report)
print("------------------------------------------")

print("data augmented with our model\n")
print(NB_augmented_report)

base

              precision    recall  f1-score   support

       dunya       0.67      0.76      0.71      1489
     ekonomi       0.84      0.37      0.52       903
        spor       1.00      0.03      0.06       298
     siyaset       0.97      0.18      0.31       373
   teknoloji       0.93      0.19      0.32       498
      kultur       0.63      0.97      0.76      2729
      saglik       1.00      0.01      0.03       216

    accuracy                           0.66      6506
   macro avg       0.86      0.36      0.39      6506
weighted avg       0.74      0.66      0.60      6506

------------------------------------------
data augmented with baseline gpt2

              precision    recall  f1-score   support

       dunya       0.76      0.67      0.71      1489
     ekonomi       0.52      0.79      0.63       903
        spor       1.00      0.02      0.03       298
     siyaset       1.00      0.10      0.18       373
   teknoloji       0.94      0.15      0.26     